In [26]:
import pycurl
import csv
import urllib
import pandas as pd

In [12]:
def csvDownload(file_name, file_src):

    with open(file_name, 'w+') as f:
        c = pycurl.Curl()
        c.setopt(c.URL, file_src)
        c.setopt(c.WRITEDATA, f.write)
        c.setopt(c.HTTPHEADER, headers)
        c.perform()
        c.close()

        f.seek(0)
        ids = find_bugs(f.readlines())

In [60]:
states=["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN",\
        "IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV",\
        "NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN",\
        "TX","UT","VT","VA","WA","WV","WI","WY"]
li = []

for state in states:
    for session in ['S1', 'S2']:
        filename='./raw_contributions/'+state+session+'.csv'
        url='https://www.opensecrets.org/races/contributors.csv?cycle=2022&id='+state+session+'&spec=N'
        #csvDownload(filename, url)
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)


contributors = pd.concat(li, axis=0, ignore_index=True)

# drop all contributions outside cycle 2022
contributors_2022 = contributors[contributors['cycle'] == 2022]

# drop unnecessary columns
contributors_2022 = contributors_2022.drop(['affiliateid', 'orgid', 'cycle',\
                                            'CID', 'Indivs', 'Pacs', 'Rank',\
                                            'CRPICO', 'ReleaseDate', 'NetRcpts'], axis=1)

# strip candidate party from name
contributors_2022['FirstLastP'] = contributors_2022['FirstLastP'].str[:-4]

# change candidate column name
contributors_2022.rename(columns = {'FirstLastP':'Candidate'}, inplace = True)

# save dataframe to pickle
contributors_2022.to_pickle('contributors_2022.pkl')

In [61]:
contributors_2022.head()

,pacid,Candidate,Contrib,Total
43,C00797670,Katie Britt,American Israel Public Affairs Cmte,99800
44,NaN,Katie Britt,Harbert Management ...,73500
45,NaN,Katie Britt,Great Southern Wood Preserving,66700
46,NaN,Katie Britt,"Beasley, Allen et al",62800
47,NaN,Katie Britt,Southern Co,57130
